# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [2]:
Summaries_file = 'data/emotion_Summaries.pkl.bz2'
Abstracts_file = 'data/emotion_Abstracts.pkl.bz2'

In [3]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [4]:
Summaries[34648542]

paper(title='Electrophysiological correlates of interference control in the modified emotional Stroop task with emotional stimuli differing in valence, arousal, and subjective significance.', authors=['Imbir KK', 'Pastwa M', 'Duda-Goławska J', 'Sobieszek A', 'Jankowska M', 'Modzelewska A', 'Wielgopolan A', 'Żygierewicz J'], year=2021, doi='10.1371/journal.pone.0258177')

In [5]:
Abstracts[33741990]

'We present the Amsterdam Open MRI Collection (AOMIC): three datasets with multimodal (3\u2009T) MRI data including structural (T1-weighted), diffusion-weighted, and (resting-state and task-based) functional BOLD MRI data, as well as detailed demographics and psychometric variables from a large set of healthy participants (N\u2009=\u2009928, N\u2009=\u2009226, and N\u2009=\u2009216). Notably, task-based fMRI was collected during various robust paradigms (targeting naturalistic vision, emotion perception, working memory, face perception, cognitive conflict and control, and response inhibition) for which extensively annotated event-files are available. For each dataset and data modality, we provide the data in both raw and preprocessed form (both compliant with the Brain Imaging Data Structure), which were subjected to extensive (automated and manual) quality control. All data is publicly available from the OpenNeuro data sharing platform.'

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [6]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [7]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(34648542)
display_summary(34648542, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [8]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'amsterdam':

In [9]:
print(inverted_index['amsterdam'])

[9756244, 16916567, 21859206, 25186285, 26784347, 29218587, 29406610, 33741990]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'utrecht':

In [1]:
query_word = 'utrecht'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

NameError: name 'inverted_index' is not defined

----------

# Tasks

**Your name:** Astha Patel

### Task 1

Implement the function `and_merge` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already, and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [11]:
def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    for i in list1:
        for j in list2:
            if i == j and i not in merged_list:
                merged_list.append(i)
    return merged_list

# testing
lst1 = [1,6,7,1,4,6,2,5,5,8]
lst2 = [1,5,7,1,3,6,2,3,6,8]
print(and_merge(lst1,lst2))

[1, 6, 7, 2, 5, 8]


### Task 2

Similarly as above, implement the function `or_merge` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [12]:
def or_merge(sorted_list1, sorted_list2):
    merged_list = []
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    list3 = list1+list2
    merged_list = list(dict.fromkeys(list3))
    return merged_list

# testing:
lst1 = [1,2,3,3]
lst2 = [4,5,6,6]
print(or_merge(lst1,lst2))

[1, 2, 3, 4, 5, 6]


### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `and_merge` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [28]:
def and_query(string):
    l1 = preprocess(tokenize(string))
    r = []
    for i in l1:
        if l1.index(i) != 0:
    r = and_merge(r, inverted_index[i])
        else:
            r = inverted_index[i]
    return r
    
        
and_query("automated quality control")


[34410788]

### Task 4

Construct another function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `or_merge` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [20]:
def or_query(string):
    l1 = preprocess(tokenize(string))
    r = []
    for i in l1:
        if l1.index(i) != 0:
            r = or_merge(r, inverted_index[i])
        else:
            r = inverted_index[i]
    return r
    
        
or_query('ASS @ CODE')


[33488995,
 25425012,
 3997585,
 7270111,
 7690981,
 9112614,
 9119582,
 9309949,
 9589285,
 11419222,
 12477605,
 12887631,
 16414281,
 17180160,
 18000554,
 19221452,
 19485687,
 20346609,
 20515223,
 20733344,
 21610856,
 21779754,
 21924368,
 21934100,
 22309729,
 22457894,
 22557958,
 23055158,
 24065950,
 24495430,
 24920615,
 25064061,
 25297056,
 25429141,
 25806706,
 25816373,
 25891321,
 26558785,
 26651622,
 26778369,
 27318957,
 27378986,
 28668136,
 28776466,
 29170615,
 29258849,
 29541041,
 29675706,
 29720691,
 29741242,
 30120839,
 30649979,
 30661754,
 30919792,
 30974241,
 31016215,
 31153307,
 31288695,
 31993472,
 32107980,
 32126893,
 32416601,
 32530313,
 32697723,
 32962172,
 33159987,
 33177977,
 33609185,
 33672413,
 33928872,
 34055236]

### Task 5

Why does `and_query('emotional factor experiment')` not return our example paper 34648542, even though it mentions emotional factors and experiments in the abstract? (You do not have to implement anything to fix this yet!)

In [24]:
Abstracts[34410788]

"Expressed emotion (EE), a measure of the family's emotional climate, is a fundamental measure in caregiving research. A core dimension of EE is the level of criticism expressed by the caregiver to the care recipient, with a high level of criticism a marker of significant distress in the household. The Five-Minute Speech Sample (FMSS), the most commonly used brief measure of EE, requires time-consuming manual processing and scoring by a highly trained expert. In this study, we used natural language processing and supervised machine learning techniques to develop a fully automated framework to evaluate caregiver criticism level based on the verbatim transcript of the FMSS. The success of the machine learning algorithm was established by demonstrating that the classification of maternal caregivers as high versus low EE was consistent with the classification of these 298 maternal caregivers of adult children with schizophrenia using standard manual coding procedures, with area under the r

**Answer:** [The only noticeable difference between the query and abstract is that the word factor is only present in the plural form. Therefore, 'factor' explcitly does not exist within the abstract.]

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.